In [24]:
using Oceananigans
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries

In [25]:
Lx = 1000kilometers # north-south extent [m]
Lz = 1kilometers    # depth [m]

grid = RectilinearGrid(size = (48, 8),
                       x = (-Lx/2, Lx/2),
                       z = (-Lz, 0),
                       topology = (Bounded, Flat, Bounded))

                       

48×1×8 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── Bounded  x ∈ [-500000.0, 500000.0] regularly spaced with Δx=20833.3
├── Flat y                             
└── Bounded  z ∈ [-1000.0, 0.0]        regularly spaced with Δz=125.0

In [26]:
α = 0  # strain rate
f = 1e-4  # Coriolis parameter [s⁻¹] at 45° latitude
δ = α/f    # deformation ratio (non-dimensional)

@inline u_deformation(x, z, t) = -α * x  # Convergent flow U = -αx
@inline w_deformation(x, z, t) =  α * z  # Compensating vertical motion

# Pack into NamedTuple for model forcing
deformation_forcing = (u=Forcing(u_deformation),
                       w=Forcing(w_deformation))

(u = ContinuousForcing{Nothing}
├── func: u_deformation (generic function with 1 method)
├── parameters: nothing
└── field dependencies: (), w = ContinuousForcing{Nothing}
├── func: w_deformation (generic function with 1 method)
├── parameters: nothing
└── field dependencies: ())

In [32]:
model = NonhydrostaticModel(; grid,
                coriolis = BetaPlane(latitude = -45),
                buoyancy = BuoyancyTracer(),
                tracers = :b,
                advection = WENO(),
                forcing = deformation_forcing)


NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 48×1×8 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO(order=5)
├── tracers: b
├── closure: Nothing
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: BetaPlane{Float64}(f₀=-0.000103126, β=1.61868e-11)

In [ ]:
# Initial condition setup (similar to your code but adjusted for frontogenesis)
N² = 1e-5  # [s⁻²] background stratification
M² = 1e-7  # [s⁻²] horizontal buoyancy gradient (front strength)

Δx = 100kilometers  # initial front width
Δb = Δx * M²       # buoyancy jump across front
ϵb = 1e-2 * Δb     # noise amplitude

ramp(x, Δx) = min(max(0, x/Δx + 1/2), 1)
bᵢ(x, z) = N² * z + Δb * ramp(x, Δx) + ϵb * randn()


bᵢ (generic function with 1 method)